<img src="http://mashey.io/wp-content/uploads/2016/02/Mashey-Logo.jpg">

# Exploring the Business Value of Churn Prediciton and Customer Retention
## Samuel Sherman

### Customer Acquisition

"Customer acquisition management is the set of methodologies and systems to manage customer prospects and inquiries generated by a variety of marketing techniques."

### Customer Retention

"Customer retention is the activity that a selling organization undertakes in order to reduce customer defections. Successful customer retention starts with the first contact an organization has with a customer and continues throughout the entire lifetime of a relationship."

### Where should a company invest the majority of their efforts?

In order to answer this question, we need to examine the cost of each and whether that will outweigh benefits.

<img src="http://sellup.net/wp-content/uploads/2016/03/Screen-Shot-2016-03-15-at-10.40.25-AM.png">

The majority of companies invest in customer acquisition, despite having higher costs. The effort to acquire one new customer can cost up to 5 times as much as an effort to retain an existing customer.

<img src="http://blog.web-media.co.uk/wp-content/uploads/2015/03/existing-new-customer.png">

For products and services offered, a company expect to gain new sales with a higher probability from existing customers than a brand new customer. However, this is a double edged sword. Your customers need to be happy with their current services in order to consider upselling. Let's examine why a customer might leave the products and services offered by a company.  

<img src="http://www.r-2w.com/blog/wp-content/uploads/2010/10/Picture-2.png">

There is clearly some value in efforts to improve customer satisfaction and retention. Now that I have made my case for customer retention, I am going to apply a use case scenario in the field of data science for churn prediction.

In [71]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.metrics import confusion_matrix
from sklearn.metrics import recall_score, precision_score, average_precision_score, roc_curve, roc_auc_score
from sklearn.linear_model import LogisticRegression as LR
from sklearn.ensemble import RandomForestClassifier as RF
from sklearn.ensemble import GradientBoostingClassifier as GBC
from sklearn.svm import SVC
from sklearn.cross_validation import cross_val_score, StratifiedShuffleSplit, StratifiedKFold, train_test_split
from sklearn.preprocessing import StandardScaler
from unbalanced_dataset.over_sampling import SMOTE
from unbalanced_dataset.under_sampling import UnderSampler
from scipy import interp
from scipy.io.arff import loadarff
%matplotlib inline


def prepare_data(filename):
    churn = loadarff(filename)
    churn_df = pd.DataFrame(churn[0])
    
    # Clean up categorical columns
    churn_df['LEAVE'] = (churn_df['LEAVE'] == 'LEAVE').astype(int)
    churn_df['COLLEGE'] = (churn_df['COLLEGE'] == "one").astype(int)
    churn_df = pd.concat([churn_df,pd.get_dummies(churn_df.REPORTED_SATISFACTION)], axis = 1)
    churn_df.drop('avg', axis = 1, inplace = True)
    churn_df = pd.concat([churn_df,pd.get_dummies(churn_df.REPORTED_USAGE_LEVEL)], axis = 1)
    churn_df.drop('avg', axis = 1, inplace = True)
    churn_df = pd.concat([churn_df,pd.get_dummies(churn_df.CONSIDERING_CHANGE_OF_PLAN)], axis = 1)
    churn_df.drop('never_thought', axis = 1, inplace = True)
    churn_df.drop('REPORTED_SATISFACTION', axis = 1, inplace = True)
    churn_df.drop('REPORTED_USAGE_LEVEL', axis = 1, inplace = True)
    churn_df.drop('CONSIDERING_CHANGE_OF_PLAN', axis = 1, inplace = True)
    
    # set label array
    y = churn_df.pop('LEAVE').values
    
    # set feature matrix
    X = churn_df.values
    
    # Scale data
    scaler = StandardScaler()
    X = scaler.fit_transform(X)
    
    return X, y, churn_df

Here, I am defining a function to prepare the dataset to be modeled. Basically, I am taking any predictive or independent variables and representing them as a 2d array or matrix. For categorical variables with more than one class, each class will need to be pivoted out and represented as TRUE or FALSE (1 or 0). Categorical variables with only two classes need to be represented as either 1 or 0 or TRUE or FALSE. Continuous variables can be left as is. Additionally, I extract out the dependent or response variable as a 1d array with values TRUE or FALSE. To be clear, TRUE and FALSE are specific class types in python that are equivalent to 1 or 0.

In [27]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.metrics import confusion_matrix
from sklearn.metrics import recall_score, precision_score, average_precision_score, roc_curve, roc_auc_score
from sklearn.linear_model import LogisticRegression as LR
from sklearn.ensemble import RandomForestClassifier as RF
from sklearn.ensemble import GradientBoostingClassifier as GBC
from sklearn.svm import SVC
from sklearn.cross_validation import cross_val_score, StratifiedShuffleSplit, StratifiedKFold, train_test_split
from sklearn.preprocessing import StandardScaler
from unbalanced_dataset.over_sampling import SMOTE
from unbalanced_dataset.under_sampling import UnderSampler
from scipy import interp
from scipy.io.arff import loadarff
%matplotlib inline


def prepare_data(filename):
    churn_df = pd.read_csv(filename)
    
    # Clean up categorical columns
    churn_df['Churn?'] = (churn_df['Churn?'] == 'True.').astype(int)
    yes_no_cols = ["Int'l Plan", "VMail Plan"]
    churn_df[yes_no_cols] = (churn_df[yes_no_cols] == "yes").astype(int)
    
    # set label array
    y = churn_df.pop('Churn?').values
    
    # Drop unwanted columns
    churn_df = churn_df.drop(['State', 'Area Code', 'Phone'], axis=1)
    
    # set feature matrix
    X = churn_df.values
    
    # Scale data
    scaler = StandardScaler()
    X = scaler.fit_transform(X)
    
    return X, y, churn_df

In [74]:
def run_logistic_churn(X, y):
    AUC, AUC2, thresholds, recall, precision = [], [], [], [], []
    mean_tpr = 0.0
    mean_fpr = np.linspace(0, 1, 100)
    
    #Statified K fold
    skf = StratifiedKFold(y, n_folds=5, shuffle=True)
    for train_index, test_index in skf:
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test= y[train_index], y[test_index]
        
        #Oversampling of unbalanced dataset
        #sm = SMOTE(kind = 'regular', verbose = True)
        #X_train, y_train = sm.fit_transform(X_train, y_train)
        #X_train, y_train = sm.fit_transform(X_train, y_train)
        
        #Undersampling of unbalanced dataset
        #u = UnderSampler()
        #X_train, y_train = u.fit_transform(X_train, y_train)
        
        # Initialize a classifier 
        clf = LR(random_state = 2, n_jobs = -1)
        clf.fit(X_train, y_train)
        pred = clf.predict_proba(X_test)
        pred2 = clf.predict(X_test)
        
        #Evaluate
        fpr, tpr, thresholds = roc_curve(y_test, pred[:,1])
        mean_tpr += interp(mean_fpr, fpr, tpr)
        mean_tpr[0] = 0.0
        AUC.append(roc_auc_score(y_test, pred[:,1]))
        AUC2.append(average_precision_score(y_test, pred[:,1]))
        recall.append(recall_score(y_test, pred2))
        precision.append(precision_score(y_test, pred2))

    mean_tpr /= len(skf)
    mean_tpr[-1] = 1.0
    return recall, AUC, precision, AUC2, mean_fpr, mean_tpr, thresholds, pred2, y_test, clf

Next, I define a function to apply a logistic regression model with stratified k-fold cross validation. I derive specific metrics for recall, precision, area under the ROC curve, and area under the precision-recall curve.

In [75]:
import plotly.plotly as py
import plotly.graph_objs as go
import plotly.tools as tls
from plotly.tools import FigureFactory as FF 
py.sign_in('scsherm', 'ml0wer7f1s')

X, y, churn_df = prepare_data('data/churn.arff')
(log_recall, log_AUC, log_precision, log_AUC2, log_mean_fpr, log_mean_tpr, 
 log_thresholds, log_pred2, log_y_test, log_clf) = run_logistic_churn(X, y)

df = pd.DataFrame(churn_df.columns.values)
df['Beta_coefficients'] = np.exp(log_clf.coef_[0])
df.rename(columns = {0:'Feature'}, inplace = True)

table_churn = FF.create_table(df)
py.iplot(table_churn, filename='coef_table_churn')

In [76]:
v = np.linspace(0,1)

data = [go.Scatter(x = log_mean_fpr, y = log_mean_tpr, 
                   mode = 'lines', 
                   name = 'ROC'),
        go.Scatter(x = v, y = v, 
                   mode = 'lines', 
                   name = '50/50 mark')]

layout = go.Layout(title='Logistic Regression ROC - AUC = {}'.format(np.mean(log_AUC)),
    xaxis=dict(title='False Postive Rate',titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f')),
    yaxis=dict(title='True Postive Rate',titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f')))
LOG_ROC_Churn = go.Figure(data=data, layout=layout)    
py.iplot(LOG_ROC_Churn, filename='LOG_ROC_Churn')

In [77]:
np.mean(log_recall), np.mean(log_AUC), np.mean(log_precision), np.mean(log_AUC2)

(0.61530702804884019,
 0.69474837249117405,
 0.64051475854377904,
 0.67995974683914839)

In [78]:
def run_rf_churn(X, y):
    AUC, AUC2, thresholds, recall, precision = [], [], [], [], []
    mean_tpr = 0.0
    mean_fpr = np.linspace(0, 1, 100)
    
    #Statified K fold
    skf = StratifiedKFold(y, n_folds=5, shuffle=True)
    for train_index, test_index in skf:
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test= y[train_index], y[test_index]
        
        #Oversampling of unbalanced dataset
        #sm = SMOTE(kind = 'regular', verbose = True)
        #X_train, y_train = sm.fit_transform(X_train, y_train)
        #X_train, y_train = sm.fit_transform(X_train, y_train)
        
        #Undersampling of unbalanced dataset
        #u = UnderSampler()
        #X_train, y_train = u.fit_transform(X_train, y_train)
        
        # Initialize a classifier 
        clf = RF(random_state = 2, n_estimators = 100, n_jobs = -1)
        clf.fit(X_train, y_train)
        pred = clf.predict_proba(X_test)
        pred2 = clf.predict(X_test)
        
        #Evaluate
        fpr, tpr, thresholds = roc_curve(y_test, pred[:,1])
        mean_tpr += interp(mean_fpr, fpr, tpr)
        mean_tpr[0] = 0.0
        AUC.append(roc_auc_score(y_test, pred[:,1]))
        AUC2.append(average_precision_score(y_test, pred[:,1]))
        recall.append(recall_score(y_test, pred2))
        precision.append(precision_score(y_test, pred2))

    mean_tpr /= len(skf)
    mean_tpr[-1] = 1.0
    return recall, AUC, precision, AUC2, mean_fpr, mean_tpr, thresholds, pred2, y_test

In [79]:
(rf_recall, rf_AUC, rf_precision, rf_AUC2, rf_mean_fpr, 
 rf_mean_tpr, rf_thresholds, rf_pred2, rf_y_test) = run_rf_churn(X, y)

v = np.linspace(0,1)

data = [go.Scatter(x = rf_mean_fpr, y = rf_mean_tpr, 
                   mode = 'lines', 
                   name = 'ROC'),
        go.Scatter(x = v, y = v, 
                   mode = 'lines', 
                   name = '50/50 mark')]

layout = go.Layout(title='Random Forest ROC - AUC = {}'.format(np.mean(rf_AUC)),
    xaxis=dict(title='False Postive Rate',titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f')),
    yaxis=dict(title='True Postive Rate',titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f')))
RF_ROC_Churn = go.Figure(data=data, layout=layout)    
py.iplot(RF_ROC_Churn, filename='RF_ROC_Churn')

In [80]:
np.mean(rf_recall), np.mean(rf_AUC), np.mean(rf_precision), np.mean(rf_AUC2)

(0.70970400760262387,
 0.75817368190627099,
 0.67898533373443859,
 0.71813651909465792)

In [81]:
from sklearn.grid_search import GridSearchCV

def run_gradient_boosted_gsearch(X, y):
    AUC, AUC2, thresholds, recall, precision = [], [], [], [], []
    mean_tpr = 0.0
    mean_fpr = np.linspace(0, 1, 100)
    
    #Statified K fold
    skf = StratifiedKFold(y, n_folds = 5, shuffle = True)
    for train_index, test_index in skf:
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test= y[train_index], y[test_index]
        
        #Oversampling of unbalanced dataset
        #sm = SMOTE(kind = 'regular', verbose = True)
        #X_train, y_train = sm.fit_transform(X_train, y_train)
        #X_train, y_train = sm.fit_transform(X_train, y_train)
        
        #Undersampling of unbalanced dataset
        #u = UnderSampler()
        #X_train, y_train = u.fit_transform(X_train, y_train)
        
        #setup paramgrid for grid search
        #param_grid = [{'learning_rate': [.01], 'n_estimators': [1000], 'max_depth': [6]}]
        clf = GBC(learning_rate = .01, n_estimators = 1000, max_depth = 6)
        #clf = GridSearchCV(GB, param_grid, verbose = 2, cv = 2, n_jobs = -1)
        clf.fit(X_train, y_train)
        pred = clf.predict_proba(X_test) 
        pred2 = clf.predict(X_test)
        
        #Evaluate
        #params.append(clf.best_params_)
        fpr, tpr, thresholds = roc_curve(y_test, pred[:,1])
        mean_tpr += interp(mean_fpr, fpr, tpr)
        mean_tpr[0] = 0.0
        AUC.append(roc_auc_score(y_test, pred[:,1]))
        AUC2.append(average_precision_score(y_test, pred[:,1]))
        recall.append(recall_score(y_test, pred2))
        precision.append(precision_score(y_test, pred2))

    mean_tpr /= len(skf)
    mean_tpr[-1] = 1.0
    return recall, AUC, precision, AUC2, mean_fpr, mean_tpr, thresholds, pred2, y_test

In [82]:
(gb_recall, gb_AUC, gb_precision, gb_AUC2, gb_mean_fpr, 
 gb_mean_tpr, gb_thresholds, gb_pred2, gb_y_test) = run_gradient_boosted_gsearch(X, y)

v = np.linspace(0,1)

data = [go.Scatter(x = gb_mean_fpr, y = gb_mean_tpr, 
                   mode = 'lines', 
                   name = 'ROC'),
        go.Scatter(x = v, y = v, 
                   mode = 'lines', 
                   name = '50/50 mark')]

    
layout = go.Layout(title='Gradient Boosted ROC - AUC = {}'.format(np.mean(gb_AUC)),
    xaxis=dict(title='False Postive Rate',titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f')),
    yaxis=dict(title='True Postive Rate',titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f')))
GB_ROC_Churn = go.Figure(data=data, layout=layout)    
py.iplot(GB_ROC_Churn, filename='GB_ROC_Churn')

In [83]:
np.mean(gb_recall), np.mean(gb_AUC), np.mean(gb_precision), np.mean(gb_AUC2)

(0.73081972870582834,
 0.76830955838739412,
 0.67669195781291192,
 0.72874935028349763)

In [99]:

log_cfmat = confusion_matrix(log_y_test,log_pred2)
rf_cfmat = confusion_matrix(rf_y_test,rf_pred2)
gb_cfmat = confusion_matrix(gb_y_test,gb_pred2)

trace1 = go.Heatmap(x=['0', '1'],y=['0', '1'],z=log_cfmat,autocolorscale=False,
    colorscale=[[0, 'rgb(220,220,220)'], [0.2, 'rgb(245,195,157)'], [0.4, 'rgb(245,160,105)'], [1, 'rgb(178,10,28)']],
    name='Trace 0, y; Trace 1, y; Trace 2, y',uid='eef847',xaxis='x',xsrc='scsherm:115:a351a4',yaxis='y',
    ysrc='scsherm:115:da265a',zauto=False,zmax=1500,zmin=400,zsrc='scsherm:115:c72e21,6fdddc')
trace2 = go.Heatmap(x=['0', '1'],y=['0', '1'],z=rf_cfmat,autocolorscale=False,
    colorscale=[[0, 'rgb(220,220,220)'], [0.2, 'rgb(245,195,157)'], [0.4, 'rgb(245,160,105)'], [1, 'rgb(178,10,28)']],
    name='Trace 0, y; Trace 1, y; Trace 2, y',uid='08b7a9',xaxis='x2',xsrc='scsherm:115:a351a4',yaxis='y2',
    ysrc='scsherm:115:da265a',zmax=1500,zmin=400,zsrc='scsherm:115:e30fc6,d7f25c')
trace3 = go.Heatmap(x=['0', '1'],y=['0', '1'],z=gb_cfmat,autocolorscale=False,
    colorscale=[[0, 'rgb(220,220,220)'], [0.2, 'rgb(245,195,157)'], [0.4, 'rgb(245,160,105)'], [1, 'rgb(178,10,28)']],
    name='Trace 0, y; Trace 1, y; Trace 2, y',uid='acf982',xaxis='x3',xsrc='scsherm:115:a351a4',yaxis='y3',
    ysrc='scsherm:115:da265a',zauto=False,zmax=1500,zmin=400,zsrc='scsherm:115:996093,c0fe9a')
data = go.Data([trace1, trace2, trace3])

layout = go.Layout(annotations=go.Annotations([go.Annotation(x=0,y=1,font=go.Font(color='rgb(255, 255, 255)',
                family='Roboto, sans-serif',size=14),showarrow=False,text=str(log_cfmat[1][0])),
        go.Annotation(x=0,y=0,font=go.Font(color='rgb(255, 255, 255)',
                family='Roboto, sans-serif',size=14),showarrow=False,text=str(log_cfmat[0][0])),
        go.Annotation(x=1,y=0,font=go.Font(color='rgb(255, 255, 255)',
                family='Roboto, sans-serif',size=14),showarrow=False,text=str(log_cfmat[0][1])),
        go.Annotation(x=1,y=1,font=go.Font(color='rgb(255, 255, 255)',
                family='Roboto, sans-serif',size=14),showarrow=False,text=str(log_cfmat[1][1])),
        go.Annotation(x=0,y=0,font=go.Font(color='rgb(255, 255, 255)',
                family='Roboto, sans-serif',size=14),showarrow=False,text=str(rf_cfmat[0][0]),xref='x2',yref='y2'),
        go.Annotation(x=0,y=1,font=go.Font(color='rgb(255, 255, 255)',
                family='Roboto, sans-serif',size=14),showarrow=False,text=str(rf_cfmat[1][0]),xref='x2',yref='y2'),
        go.Annotation(x=1,y=0,font=go.Font(color='rgb(255, 255, 255)',
                family='Roboto, sans-serif',size=14),showarrow=False,text=str(rf_cfmat[0][1]),xref='x2',yref='y2'),
        go.Annotation(x=1,y=1,font=go.Font(color='rgb(255, 255, 255)',
                family='Roboto, sans-serif',size=14),showarrow=False,text=str(rf_cfmat[1][1]),xref='x2',yref='y2'),
        go.Annotation(x=0,y=0,font=go.Font(color='rgb(255, 255, 255)',
                family='Roboto, sans-serif',size=14),showarrow=False,text=str(gb_cfmat[0][0]),xref='x3',yref='y3'),
        go.Annotation(x=0,y=1,font=go.Font(color='rgb(255, 255, 255)',
                family='Roboto, sans-serif',size=14),showarrow=False,text=str(gb_cfmat[1][0]),xref='x3',yref='y3'),
        go.Annotation(x=1,y=0,font=go.Font(color='rgb(255, 255, 255)',
                family='Roboto, sans-serif',size=14),showarrow=False,text=str(gb_cfmat[0][1]),xref='x3',yref='y3'),
        go.Annotation(x=1,y=1,font=go.Font(color='rgb(255, 255, 255)',
                family='Roboto, sans-serif',size=14),showarrow=False,text=str(gb_cfmat[1][1]),xref='x3',yref='y3')]),
    height=600,
    title='Confusion Matrices',
    width=1000,
    xaxis=go.XAxis(title = 'Logistic - Predicted', anchor='y',autorange=True,
                   domain=[0, 0.2888888888888889],range=[-0.5, 1.5],type='category'),
    xaxis2=go.XAxis(title = 'Random Forest - Predicted', anchor='y2',autorange=True,
                    domain=[0.35555555555555557, 0.6444444444444445],range=[-0.5, 1.5], type='category'),
    xaxis3=go.XAxis(title = 'Gradient Boosted - Predicted', anchor='y3',autorange=True,domain=[0.7111111111111111, 1],
                    range=[-0.5, 1.5],type='category'),
    yaxis=go.YAxis(anchor='x',autorange=True,domain=[0, 1],range=[-0.5, 1.5],title='Actual Class',type='category'),
    yaxis2=go.YAxis(anchor='x2',autorange=True,domain=[0, 1],range=[-0.5, 1.5],type='category'),
    yaxis3=go.YAxis(anchor='x3',autorange=True,domain=[0, 1],range=[-0.5, 1.5],type='category'))
fig = go.Figure(data=data, layout=layout)
py.iplot(fig)

It is clear that the gradient boosted model is performing best. The total accuracy, as well as true positive count is higher than the other models. The total count of false labels is less than the other models. The logistic regression model has a large spread throught the falsely classified labels. The random forest model is performing similarly, but the gradient boosted model has a higher count for true positives and thus a higher recall. 

In [98]:
from IPython.core.display import HTML
import urllib2
HTML(urllib2.urlopen('http://bit.ly/1Bf5Hft').read())